Load Packages

In [27]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model

# Import NLTK #
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import nltk

# Import RE #
import re

# Set Seed so that the result is consistent
np.random.seed(1)


### Meta Settings

-license-

@InProceedings{Pang+Lee:04a,
  author =       {Bo Pang and Lillian Lee},
  title =        {A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts},
  booktitle =    "Proceedings of the ACL",
  year =         2004
}

In [198]:
data_dir = 'data/'
sub_file_name = data_dir + 'quote.tok.gt9.5000'
obj_file_name = data_dir + 'plot.tok.gt9.5000'
#---#
sub_dict_name = data_dir + 'subjectivity.lexicon.json'

### Load Dataset and Represent

In [33]:
# Load Subjective Sentences Set
sub_file = open(sub_file_name, 'r', encoding = "ISO-8859-1")
sub_text = sub_file.read()
sub_list = sub_text.split('\n')
sub_file.close()

# Load Objective Sentences Set
obj_file = open(obj_file_name, 'r', encoding = "ISO-8859-1")
obj_text = obj_file.read()
obj_list = obj_text.split('\n')
obj_file.close()


Turn sentences to vectors

1. 形容詞數量(Number of ajectives)
2. 名詞數量(Number of nouns)
3. 字數(Word Count), excluding stop words and punctuation
4. 出現在字典強烈的主觀字的字數(Number of strongly subjective words) PS, 這邊用兩個subjective words dictionary, SentiWordNet, the Subjectivity Lexicon, 兩者各為一個feature
5. Name Enitity的數量(Number of Name Entity)
6. 比較級字數(Number of comparative)
7. 最高級字數(Number of superlative)
8. 時態(tense)
9. 副詞字數(Number of Adverbs)
10. 時間、日期、數字的出現次數(Number of date, time, numbers)

O 1,2,3,4,5,6,7,9,10
X 8


In [34]:
# Define Pattern

# Get Adjective: JJ
PATTERN_ADJ = re.compile('JJ.*')

## Get Comparative: JJR, RBR
PATTERN_COMP = re.compile('JJR|RBR')

## Get Superlative: JJS, RBS
PATTERN_SUP = re.compile('JJS|RBS')

# Get Nouns: NN
PATTERN_NOUN = re.compile('NN.*')

# Get Adverb: RB
PATTERN_ADV = re.compile('RB.*')

# Get Date and Number: CD
PATTERN_NUM = re.compile('CD')



In [61]:
# Calculate Num of Name Entity
def numOfNE(sent):
    parse_tree = ne_chunk(pos_tag(word_tokenize(sent)), binary=True)
    named_entities = []
    for t in parse_tree.subtrees():
        if t.label() == 'NE':
            named_entities.append(t.leaves())
            # named_entities.append(list(t))  # if you want to save a list of tagged words instead of a tree

    return len(named_entities)


3


這裡可能產生一些問題
因為照理說，我應該要用該詞性下的分數去算（或許能解決）
再來，如果是一個詞組，怎麼算（可能兩個字分開是正面意思，合起來變成負面）

In [194]:
# Use SentiWordNet
from nltk.corpus import sentiwordnet as swn

# The objectivity score can be calculated as:
# ObjScore = 1 - (PosScore + NegScore)

def scoreOfSentiWordNet(sent_list):
    total_score = 0
    cnt = 0
    for word in sent_list:
        w = swn.senti_synsets(word)
        w_l = list(w)
        if len(w_l) != 0:
            w0 = w_l[0]
            obj_score = 1 - (w0.pos_score() + w0.neg_score())
            if obj_score < 1.00:
                total_score = total_score + obj_score
                cnt = cnt + 1
#
#            else:
#                print("objective word: {}, score={}".format(word, obj_score))
#        else:
#            print("not here: " + word)
    # It would be better to do average
    if cnt != 0:
        total_score = (total_score / cnt)
    else:
        total_score = 1
        
    # For test
#    if total_score < 0.5:
#        print(sent_list)
    return total_score


接下來是要用the subjectivity Lexicon
如果土法煉鋼的做
就是先讀JSON存成字典
然後接下來每個字去查字典裡有沒有
有的話再看它的type是什麼
基本上只有strongsubj跟weaksubj
沒有在字典裡的則屬於obj

這邊，把strongsubj = 1
weaksubj = 0.5
Let's go!


In [234]:
import json
sd = open(sub_dict_name, 'r')
sub_dict = json.load(sd)
print(sub_dict[0])

def subjectivity_score(word):
    for w in sub_dict:
        if word == w['word1']:
            if w['type'] == 'weaksubj':
                return 0.5
            else:
                return 1
            break
    return 0
def subjectivity_score_sentence(sent):
    total_score = 0
    for w in sent:
        total_score = total_score + subjectivity_score(w)
    return total_score

{'type': 'weaksubj', 'len': '1', 'word1': 'abandoned', 'pos1': 'adj', 'stemmed1': 'n', 'priorpolarity': 'negative'}


In [235]:
# Turn a sentence to a vector:  {'NOUN': 0, 'ADJ': 0, 'COMP': 0, 'SUP': 0, 'ADV': 0, 'NUM': 0, 'WC': 0}
def Sent2Vec(sent):
    # Define Vector
    vec = {'NOUN': 0, 'ADJ': 0, 'COMP': 0, 'SUP': 0, 'ADV': 0, 'NUM': 0, 'WC': 0, 'NE':0, 'SentiWordNet': 0, 'Subjectivity': 0}
    # Word count exluding stopwords and punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    no_punc = tokenizer.tokenize(sent)
    no_punc_stopw = []
    for w in no_punc:
        if w not in stopwords.words():
            no_punc_stopw.append(w)
    
    # After filtering stopwords and punctuation, get word count
    vec['WC'] = len(no_punc_stopw)
    
    # Use numOfNE function
    vec['NE'] = numOfNE(sent)
    
    # Use SentiWordNet
    vec['SentiWordNet'] = scoreOfSentiWordNet(no_punc_stopw)
    
    # Use the subjectivity Lexicon
    vec['Subjectivity'] = subjectivity_score_sentence(no_punc_stopw)
    
    tagged = nltk.pos_tag(no_punc_stopw)
    tag_fd = nltk.FreqDist(tag for (word, tag) in tagged)
    
    for tag, count in tag_fd.most_common():       
        if PATTERN_ADJ.match(tag):
            vec['ADJ'] = vec['ADJ'] + count
        if PATTERN_NOUN.match(tag):
            vec['NOUN'] = vec['NOUN'] + count
        if PATTERN_COMP.match(tag):
            vec['COMP'] = vec['COMP'] + count
        if PATTERN_SUP.match(tag):
            vec['SUP'] = vec['SUP'] + count
        if PATTERN_ADV.match(tag):
            vec['ADV'] = vec['ADV'] + count
        if PATTERN_NUM.match(tag):
            vec['NUM'] = vec['NUM'] + count
    return vec

In [242]:
#--Test Sent2Vec Function--#
#!! We found that in this dateset, there are almost no Name Entity.
test_sent = obj_list[7]
cnt = 1
for sen in obj_list:
    vec = Sent2Vec(sen)
    print(vec)
    cnt = cnt + 1
    if cnt == 10:
        break
print('Done')

{'NOUN': 5, 'ADJ': 3, 'COMP': 0, 'SUP': 0, 'ADV': 0, 'NUM': 0, 'WC': 11, 'NE': 0, 'SentiWordNet': 0.7916666666666666, 'Subjectivity': 0}
{'NOUN': 12, 'ADJ': 6, 'COMP': 0, 'SUP': 0, 'ADV': 0, 'NUM': 0, 'WC': 22, 'NE': 0, 'SentiWordNet': 0.8333333333333334, 'Subjectivity': 1.0}
{'NOUN': 9, 'ADJ': 3, 'COMP': 1, 'SUP': 0, 'ADV': 0, 'NUM': 0, 'WC': 15, 'NE': 0, 'SentiWordNet': 0.8333333333333334, 'Subjectivity': 0.5}
{'NOUN': 7, 'ADJ': 0, 'COMP': 1, 'SUP': 0, 'ADV': 1, 'NUM': 0, 'WC': 14, 'NE': 0, 'SentiWordNet': 0.6071428571428571, 'Subjectivity': 6.0}
{'NOUN': 3, 'ADJ': 2, 'COMP': 0, 'SUP': 0, 'ADV': 0, 'NUM': 0, 'WC': 9, 'NE': 0, 'SentiWordNet': 0.875, 'Subjectivity': 1}
{'NOUN': 7, 'ADJ': 5, 'COMP': 0, 'SUP': 0, 'ADV': 0, 'NUM': 0, 'WC': 16, 'NE': 0, 'SentiWordNet': 0.4791666666666667, 'Subjectivity': 1.5}
{'NOUN': 7, 'ADJ': 1, 'COMP': 0, 'SUP': 0, 'ADV': 0, 'NUM': 0, 'WC': 12, 'NE': 0, 'SentiWordNet': 0.75, 'Subjectivity': 0.5}
{'NOUN': 3, 'ADJ': 2, 'COMP': 0, 'SUP': 0, 'ADV': 1, 'NUM'

In [245]:
# Turn All Data to Vectors and Store in sub_vec, obj_vec

# Verify the size
print("Notice: The Last Line is Empty")
print(len(sub_list))
print(len(obj_list))

# Define SIZE 
SIZEOFSENTENCES = 5000
SIZEOFVECTORS = 10

# Use Two Matrices to Store Vectorize Sentences
OBJ_MATRIX = np.zeros((SIZEOFSENTENCES, SIZEOFVECTORS))  
SUB_MATRIX = np.zeros((SIZEOFSENTENCES, SIZEOFVECTORS))

cnt = 0
print("Start Converting Objective Sentences:")
for sent in obj_list:
    vec = Sent2Vec(sent)
    OBJ_MATRIX[cnt] = [vec['ADJ'], vec['NOUN'], vec['WC'], vec['NUM'], vec['COMP'], vec['SUP'], vec['ADV'], vec['NE'], vec['SentiWordNet'], vec['Subjectivity']]
    cnt = cnt + 1
    if cnt % 50 == 0:
        print(str(cnt / 50) + '%')
    if cnt == 5000:
        break
cnt = 0
print("Start Converting Subjective Sentences:")
for sent in sub_list:
    vec = Sent2Vec(sent)
    SUB_MATRIX[cnt] = [vec['ADJ'], vec['NOUN'], vec['WC'], vec['NUM'], vec['COMP'], vec['SUP'], vec['ADV'], vec['NE'], vec['SentiWordNet'], vec['Subjectivity']]
    cnt = cnt + 1
    if cnt % 50 == 0:
        print(str(cnt / 50) + '%')
    if cnt == 5000:
        break
    
# Store as Files so that we could load next time
print("Start Writing to files")
np.savetxt('obj.txt', OBJ_MATRIX)
np.savetxt('sub.txt', SUB_MATRIX)
# Done
print('Done')

Notice: The Last Line is Empty
5001
5001
Start Converting Objective Sentences:
1.0%
2.0%
3.0%
4.0%
5.0%
6.0%
7.0%
8.0%
9.0%
10.0%
11.0%
12.0%
13.0%
14.0%
15.0%
16.0%
17.0%
18.0%
19.0%
20.0%
21.0%
22.0%
23.0%
24.0%
25.0%
26.0%
27.0%
28.0%
29.0%
30.0%
31.0%
32.0%
33.0%
34.0%
35.0%
36.0%
37.0%
38.0%
39.0%
40.0%
41.0%
42.0%
43.0%
44.0%
45.0%
46.0%
47.0%
48.0%
49.0%
50.0%
51.0%
52.0%
53.0%
54.0%
55.0%
56.0%
57.0%
58.0%
59.0%
60.0%
61.0%
62.0%
63.0%
64.0%
65.0%
66.0%
67.0%
68.0%
69.0%
70.0%
71.0%
72.0%
73.0%
74.0%
75.0%
76.0%
77.0%
78.0%
79.0%
80.0%
81.0%
82.0%
83.0%
84.0%
85.0%
86.0%
87.0%
88.0%
89.0%
90.0%
91.0%
92.0%
93.0%
94.0%
95.0%
96.0%
97.0%
98.0%
99.0%
100.0%
Start Converting Subjective Sentences:
1.0%
2.0%
3.0%
4.0%
5.0%
6.0%
7.0%
8.0%
9.0%
10.0%
11.0%
12.0%
13.0%
14.0%
15.0%
16.0%
17.0%
18.0%
19.0%
20.0%
21.0%
22.0%
23.0%
24.0%
25.0%
26.0%
27.0%
28.0%
29.0%
30.0%
31.0%
32.0%
33.0%
34.0%
35.0%
36.0%
37.0%
38.0%
39.0%
40.0%
41.0%
42.0%
43.0%
44.0%
45.0%
46.0%
47.0%
48.0%
49.0%
50.0%

Build Training Set and Testing Set
We put objective sentences in even entries, subjective sentences in odd entries


In [246]:
# definition
TRAINSET_SIZE = 7000
TESTSET_SIZE = 3000
DATA_SIZE = TRAINSET_SIZE + TESTSET_SIZE
SLICE_SIZE = int(TRAINSET_SIZE/2)
END_SIZE = int(DATA_SIZE / 2)

# concatenate subjective dataset and objective dataset / only 100
X = np.zeros((TRAINSET_SIZE, SIZEOFVECTORS))
Y = np.zeros((TRAINSET_SIZE, 1))
test_X = np.zeros((TESTSET_SIZE, SIZEOFVECTORS))
test_Y = np.zeros((TESTSET_SIZE, 1))

X[0:TRAINSET_SIZE+1:2] = OBJ_MATRIX[0:SLICE_SIZE]
X[1:TRAINSET_SIZE+1:2] = SUB_MATRIX[0:SLICE_SIZE]
Y[1:TRAINSET_SIZE+1:2] = 1

test_X[0:TESTSET_SIZE:2] = OBJ_MATRIX[SLICE_SIZE:END_SIZE]
test_X[1:TESTSET_SIZE+1:2] = SUB_MATRIX[SLICE_SIZE:END_SIZE]
test_Y[1:TESTSET_SIZE+1:2] = 1
# Translate Matrix
X = X.T
Y = Y.T
test_X = test_X.T
test_T = test_Y.T
print('Done')

Done


In [209]:
# Search All Posibilities of Tag
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

Implement some needed functions

In [210]:
# need to use np.exp rather than math.exp since we use np.array
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


Design the flow of DL Model

1. Define the neural network structure ( # of input units,  # of hidden units, etc). 
2. Initialize the model's parameters
3. Loop:
    - Implement forward propagation
    - Compute loss
    - Implement backward propagation to get the gradients
    - Update parameters (gradient descent)

In [211]:
#--Test--#
#--OK--#
# set the size of input layer, hidden layer, and output layer
def initialize_parameters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h, n_x)
    b1 = np.zeros(shape=(n_h, 1)) # axis = 0 need to consistent with W1
    W2 = np.random.randn(n_y, n_h)
    b2 = np.zeros(shape=(n_y, 1))
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    return parameters 

In [212]:
def forward_propagation(X, parameters):
    # Get Parameters
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Do Forward Propgation
    Z1 = np.dot(W1, X) + b1 # if there were more than one training examples, b1 using "broadcast"
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [237]:
#--Test Forward Propagation--#
#--OK--#
paras = initialize_parameters(SIZEOFVECTORS, 3, 1)
A2, cache = forward_propagation(X, paras)
print(A2, cache)

[[ 0.30701011  0.54976045  0.5606515  ...,  0.53559576  0.07322341
   0.29167066]] {'Z1': array([[ 19.01631799,  16.17187011,  40.19039549, ...,   8.64151048,
         24.51808753,  20.28627066],
       [ 13.32594952,   8.45607636,  29.54634052, ...,   7.06725703,
         15.95975178,  17.07700052],
       [  0.24366345,   2.05102906,   3.88291135, ...,   1.63269893,
         -4.94029973,   0.18859527]]), 'A1': array([[ 1.        ,  1.        ,  1.        , ...,  0.99999994,
         1.        ,  1.        ],
       [ 1.        ,  0.99999991,  1.        , ...,  0.99999855,
         1.        ,  1.        ],
       [ 0.23895303,  0.96746095,  0.9991524 , ...,  0.92644498,
        -0.99989769,  0.18639063]]), 'Z2': array([[-0.81413473,  0.19970284,  0.24380654, ...,  0.14262433,
        -2.53819732, -0.88728386]]), 'A2': array([[ 0.30701011,  0.54976045,  0.5606515 , ...,  0.53559576,
         0.07322341,  0.29167066]])}


In [215]:
def compute_loss(A2, Y, parameters):
    # Get the Num of Examples
    m = Y.shape[1]
    
    # Compute Loss
    logprobs = np.multiply(np.log(A2), Y) + np.multiply(1-Y, np.log(1 - A2)) 
    cost = - np.sum(logprobs) / m
    
    # Squeeze, make np.array to a scalar
    cost = np.squeeze(cost)
    return cost

In [216]:
#--Test Cost--#
#--OK--#
print(compute_loss(A2, Y, paras))

0.68895204018


In [217]:
def backward_propagation(parameters, cache, X, Y):
    # Retrieve W1, W2 from paras
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    # Retrieve A1, A2 from cache
    A1 = cache['A1']
    A2 = cache['A2']
    
    # Get the Num of Examples
    m = Y.shape[1]
    
    # Do Backward Propagation
    dZ2 = A2 - Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims = True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    
    # Grad
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    return grads

In [218]:
#--Test--#
#--OK--#
print(backward_propagation(paras, cache, X, Y))

{'dW1': array([[ -2.88469846e-04,  -5.38455145e-03,  -6.52692876e-03,
          2.51239025e-04,  -5.39987770e-05,  -4.94863067e-05,
          3.67164195e-05,   0.00000000e+00,  -4.52948985e-04],
       [  1.63963245e-03,   6.47654413e-04,   7.64267581e-03,
          2.48684929e-03,  -7.51495292e-06,  -8.68519532e-05,
         -1.11596867e-04,   0.00000000e+00,   4.25499455e-04],
       [  1.08319858e-03,   1.29507424e-04,   1.52377353e-03,
          1.97668800e-05,  -3.20510662e-05,  -2.97964369e-06,
          3.79403100e-04,   0.00000000e+00,  -7.57001847e-05]]), 'db1': array([[ -8.10874791e-04],
       [ -4.90203186e-04],
       [ -7.71909975e-05]]), 'dW2': array([[-0.02397239, -0.01159115, -0.03122895]]), 'db2': array([[-0.03075922]])}


In [219]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    # Retreive Parameters
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Retreive grads
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    # Update
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    # Reset Parameters, since update
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [220]:
#--Test--#
#--OK--#
# Get grads
grads = backward_propagation(paras, cache, X, Y)

# Update
n_paras = update_parameters(paras, grads)

# print paras
print(paras)
print(n_paras)

{'W1': array([[ 1.48754313, -1.11830068,  0.84583341, -1.86088953, -0.6028851 ,
        -1.91447204,  1.04814751,  1.33373782, -0.19741468],
       [ 1.77464503, -0.67472751,  0.15061687,  0.1529457 , -1.06419527,
         0.43794661,  1.93897846, -1.02493087,  0.89933845],
       [-0.15450685,  1.7696273 ,  0.48378835,  0.6762164 ,  0.64316328,
         0.24908671, -1.3957635 ,  1.39166291, -1.37066901]]), 'b1': array([[ 0.],
       [ 0.],
       [ 0.]]), 'W2': array([[ 0.23856319,  0.61407709, -0.83791227]]), 'b2': array([[ 0.]])}
{'W1': array([[ 1.4878893 , -1.11183922,  0.85366572, -1.86119102, -0.60282031,
        -1.91441266,  1.04810345,  1.33373782, -0.19687114],
       [ 1.77267747, -0.6755047 ,  0.14144565,  0.14996148, -1.06418626,
         0.43805083,  1.93911238, -1.02493087,  0.89882785],
       [-0.15580669,  1.76947189,  0.48195982,  0.67619268,  0.64320174,
         0.24909028, -1.39621879,  1.39166291, -1.37057817]]), 'b1': array([[  9.73049749e-04],
       [  5.88243

In [247]:
def nn_model(X, Y, n_h, num_iterations=10000, print_cost=False):
    # Set Random Seed to make time consistent
    np.random.seed(3)
    
    # Get n_x, n_y, and we have n_h
    n_x = X.shape[0]
    n_y = Y.shape[0]
    
    # Initialize Parameters
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Loop
    for i in range(0, num_iterations):
        # Forward Propagation
        A2, cache = forward_propagation(X, parameters)
        
        # Count Cost
        cost = compute_loss(A2, Y, parameters)
        
        # Backward Propagation
        grads = backward_propagation(parameters, cache, X, Y)
        
        # Update Parameters
        parameters = update_parameters(parameters, grads)
        
        # Print Cost
        if print_cost and i % 1000 == 0:
            print("Cost after iteration %i: %f" % (i, cost))
    return parameters
    

In [261]:
#--Test--#
#Put X and Y data in nn_model#
#--OK--#
after_paras = nn_model(X, Y, 7, 15000, True)
print(after_paras)


Cost after iteration 0: 1.001140
Cost after iteration 1000: 0.620777
Cost after iteration 2000: 0.620261
Cost after iteration 3000: 0.616564
Cost after iteration 4000: 0.614593
Cost after iteration 5000: 0.613715
Cost after iteration 6000: 0.613372
Cost after iteration 7000: 0.613249
Cost after iteration 8000: 0.613092
Cost after iteration 9000: 0.612827
{'W1': array([[ 1.78722667,  0.63721102,  1.08754754, -1.42512831, -0.24995   ,
        -0.35850623,  0.32085574, -0.62700068, -0.35752849, -1.73479982],
       [-0.92726711,  0.99183766,  1.64497578,  1.70864043,  0.05450135,
        -0.40782405, -0.36924884, -1.54647732,  1.05670824, -0.58864842],
       [ 0.38711035, -0.58764944,  1.13695366,  0.24998432, -1.98381217,
        -1.78079456,  0.81087304, -0.16051336, -1.05480142, -0.90022728],
       [ 0.64843213,  1.01156782, -2.49729368, -0.64741526, -0.80598035,
        -2.41825323, -0.92031353, -1.02387576,  0.83488546, -0.19941578],
       [-1.25091664, -0.35576077,  1.50314114, -

In [262]:
def predict(parameters, X):
    A2, cache = forward_propagation(X, parameters)
    predictions = np.round(A2)
    
    return predictions

In [263]:
#--Test--#
predictions = predict(after_paras, test_X)
print(predictions)
print("predictions mean = " + str(np.mean(predictions)))


[[ 0.  1.  0. ...,  1.  1.  1.]]
predictions mean = 0.656666666667


Calculate Recall, Precision, Accuracy, F-measure

In [264]:
# Evaluation
# tp, tn
tp = np.sum(np.dot(predictions, test_Y))
tn = np.sum(np.dot(1-predictions, 1-test_Y))

# precision
precision = tp / np.sum(predictions)
print("precision = " + str(precision))

# accuracy
accuracy = ( tp + tn ) / predictions.size
print("accuracy = " + str(accuracy))

# recall
recall = tp / np.sum(test_Y)
print("recall = " + str(recall))

# F-measure
F = (2*precision*recall) / (precision + recall)
print("F-measure = " + str(F))

precision = 0.617258883249
accuracy = 0.654
recall = 0.810666666667
F-measure = 0.700864553314


Use 10-cross validation